# Default Values — Advanced Practice (with inline tests)

Goals:
- Choose sensible defaults that make functions easy to call.
- Know when to prefer keyword-only options.
- Avoid *mutable default* traps via `None` + factory.
- Use sentinels when `None` is a valid value.
- Build small utilities that compose well.

Each problem includes **assert tests right after the function definition**. Run cell-by-cell.

## Problem 1 — `clamp(x, min_=0, max_=1)`
Return `x` limited to the inclusive range `[min_, max_]`.

Rules:
- If `min_ > max_` raise `ValueError`.
- Accept ints or floats; result type should follow float if any input is float.
- Defaults make it easy to clamp into the unit interval.

In [1]:
from typing import Union

Number = Union[int, float]

def clamp(x: Number, min_: Number = 0, max_: Number = 1) -> Number:
    """Clamp x into [min_, max_]."""
    if min_ > max_:
        raise ValueError("min_ cannot be greater than max_")
    y = x
    if y < min_:
        y = min_
    if y > max_:
        y = max_
    # preserve float if any arg is float
    if any(isinstance(v, float) for v in (x, min_, max_)):
        return float(y)
    return int(y)


In [2]:
# Tests — Problem 1
assert clamp(0.2) == 0.2
assert clamp(-3) == 0
assert clamp(10, 0, 5) == 5
assert clamp(3, 3, 3) == 3
try:
    clamp(1, 5, 2)
    raise AssertionError("Expected ValueError when min_ > max_")
except ValueError:
    pass


## Problem 2 — `is_close(a, b, abs_tol=1e-9)`
Re-implement a simple absolute-tolerance comparison with a sensible **small default**.

Rules:
- Return `True` if `|a - b| <= abs_tol`, else `False`.
- Validate `abs_tol >= 0` else `ValueError`.
- Keep signature minimal and friendly.

In [3]:
def is_close(a: Number, b: Number, abs_tol: float = 1e-9) -> bool:
    if abs_tol < 0:
        raise ValueError("abs_tol must be non-negative")
    return abs(float(a) - float(b)) <= abs_tol


In [4]:
# Tests — Problem 2
assert is_close(1.0, 1.0)
assert is_close(0.0, 1e-10)
assert not is_close(0.0, 1e-6)
assert is_close(10001, 10002, abs_tol=5)
try:
    is_close(1, 1, abs_tol=-1)
    raise AssertionError("Expected ValueError for negative abs_tol")
except ValueError:
    pass


## Problem 3 — Avoid mutable-default trap: `append_log(entry, log=None)`
Append `entry` to a list and return the list. Default should produce a **fresh list per call**.

Rules:
- If `log is None`, create a new list; else mutate the provided list.
- Return the resulting list (for easy chaining/testing).

In [5]:
from typing import List, Optional, Any

def append_log(entry: Any, log: Optional[List[Any]] = None) -> List[Any]:
    if log is None:
        log = []
    log.append(entry)
    return log


In [6]:
# Tests — Problem 3
a = append_log("start")
b = append_log("again")
assert a == ["start"] and b == ["again"]  # distinct lists
same = append_log("x", a)
assert same is a and a == ["start", "x"]


## Problem 4 — Keyword-only defaults: `split_clean(s, *, sep=",", strip=True)`
Return a list by splitting `s` on `sep`. If `strip=True`, strip whitespace from parts.

Rules:
- Options are keyword-only to make calls self-documenting.
- Keep defaults useful for CSV-like inputs.
- Return list of strings; do not drop empty items (mirror `str.split`).

In [7]:
from typing import List

def split_clean(s: str, *, sep: str = ",", strip: bool = True) -> List[str]:
    parts = s.split(sep)
    if strip:
        return [p.strip() for p in parts]
    return parts


In [8]:
# Tests — Problem 4
assert split_clean(" a , b , c ") == ["a", "b", "c"]
assert split_clean("a| b | c", sep="|") == ["a", "b", "c"]
assert split_clean("a| b | ", sep="|", strip=False) == ["a", " b ", " "]


## Problem 5 — Sentinel default when `None` is valid: `coalesce(*values, default=_MISSING)`
Return the first argument that is **not** `None`. If all values are `None` and `default` wasn't provided, raise `ValueError`.

Why sentinel? Because caller might *want* `default=None` explicitly.

Examples:
- `coalesce(None, 0, 2) -> 0`
- `coalesce(None, None, default=5) -> 5`
- `coalesce(None) -> ValueError` (since default missing)

In [9]:
_MISSING = object()

def coalesce(*values: Any, default: Any = _MISSING) -> Any:
    for v in values:
        if v is not None:
            return v
    if default is _MISSING:
        raise ValueError("All values are None and no default provided")
    return default


In [10]:
# Tests — Problem 5
assert coalesce(None, 0, 2) == 0
assert coalesce(None, None, default=5) == 5
assert coalesce(None, None, default=None) is None
try:
    coalesce(None)
    raise AssertionError("Expected ValueError when all None and no default")
except ValueError:
    pass


## Problem 6 — Safe division with configurable fallback: `safe_div(a, b=1, *, on_error=float('inf'))`
Return `a / b`. If `b == 0`, return `on_error` (default: `inf`).

Rules:
- Keep `on_error` keyword-only so callers must be explicit.
- Handle ints/floats. No broad exception swallowing — just `ZeroDivisionError`.

In [11]:
def safe_div(a: Number, b: Number = 1, *, on_error: Number = float("inf")) -> float:
    try:
        return float(a) / float(b)
    except ZeroDivisionError:
        return float(on_error)


In [12]:
# Tests — Problem 6
assert safe_div(10, 2) == 5.0
assert safe_div(1, 0) == float("inf")
assert safe_div(1, 0, on_error=-1) == -1.0


## Problem 7 — Closure factory with defaults: `make_counter(start=0, step=1)`
Return a function `counter()` that when called increments an internal value by `step` and returns it, starting from `start`.

Rules:
- Each call to `make_counter` returns an **independent** counter.
- Defaults make common counters trivial (start at 0, step by 1).

In [13]:
from typing import Callable

def make_counter(start: Number = 0, step: Number = 1) -> Callable[[], Number]:
    value = start
    def counter() -> Number:
        nonlocal value
        value = value + step
        return value
    return counter


In [14]:
# Tests — Problem 7
c1 = make_counter()
c2 = make_counter(start=10, step=5)
assert [c1(), c1(), c1()] == [1, 2, 3]
assert [c2(), c2()] == [15, 20]
c3 = make_counter()
assert c3() == 1 and c1() == 4  # independence


## Problem 8 — Pretty join with flexible defaults: `join_items(items, sep=", ", prefix="", suffix="")`
Join an iterable of items into a single string, converting each item with `str()` and adding optional `prefix`/`suffix`.

Examples:
- `join_items([1,2,3]) -> '1, 2, 3'`
- `join_items(["a","b"], sep="|", prefix="[", suffix="]") -> '[a|b]'`

Notes:
- Defaults are friendly; keyword args make formatting explicit.

In [15]:
from typing import Iterable

def join_items(items: Iterable[object], sep: str = ", ", prefix: str = "", suffix: str = "") -> str:
    body = sep.join(str(x) for x in items)
    return f"{prefix}{body}{suffix}"


In [16]:
# Tests — Problem 8
assert join_items([1,2,3]) == "1, 2, 3"
assert join_items(["a","b"], sep="|", prefix="[", suffix="]") == "[a|b]"
assert join_items([]) == ""


---
### Optional sanity sweep
Run to re-check everything at once.

In [17]:
assert clamp(2.5, 0.0, 1.0) == 1.0
assert is_close(0.0, 1e-10)
assert append_log("x") == ["x"]
assert split_clean(" a , b , c ") == ["a","b","c"]
assert coalesce(None, None, default=0) == 0
assert safe_div(1, 0, on_error=-999.0) == -999.0
cnt = make_counter(5, 2); assert [cnt(), cnt()] == [7, 9]
assert join_items([1,2], prefix="(", suffix=")") == "(1, 2)"
print("All checks passed ✔️")


All checks passed ✔️
